#### Calculate the RMSEs of CBF maps as the parameter selection standard

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import scipy.ndimage

In [2]:
sys.path.append('../Preprocess')
import CalcParaMaps
import BiasCorrection

In [27]:
imgNorm = 0.15
iTests = np.arange(55,70)
N0 = 100000

In [28]:
# load reference
with np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/paras_tikh_0.3.npz') as f:
    masks = f['mask'][iTests,...]
    cbfFac = f['cbfFac']
aif = np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/aif0.npy') / 1000 / imgNorm
refs = (np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/imgs_-1.npy')[iTests, ...] - 1) / imgNorm
imgs = (np.load('/home/dwu/trainData/Noise2Noise/train/ctp/simul/data/imgs_%g.npy'%N0)[iTests, ...] - 1) / imgNorm

In [29]:
maskVessels = np.where(np.max(imgs, -1) > 0.1 / imgNorm, 1, 0)
maskVessels *= masks
for i in range(maskVessels.shape[0]):
    maskVessels[i,...] = scipy.ndimage.morphology.binary_dilation(maskVessels[i,...])
masks *= (1-maskVessels)

refs *= masks[...,np.newaxis]
imgs *= masks[...,np.newaxis]

In [30]:
cbf0, _, _ = CalcParaMaps.CalcParaMaps(refs - refs[...,[0]], masks, kappa=1, rho=1, aif = np.copy(aif))
cbf0 *= cbfFac
ctps = imgs - (imgs[...,[0]] + imgs[...,[1]]) / 2

In [31]:
# Calc all the rmses for Noise2Noise
filenames = glob.glob('/home/dwu/trainData/Noise2Noise/train/ctp/simul/beta_*_N0_%g/tmp/iodines.npy'%N0)
for i, filename in enumerate(filenames):
    print (i, end=',')
    tac = np.load(filename)[iTests,...].transpose(0, 2, 3, 1) * 0.025 / imgNorm
    bc = BiasCorrection.BiasCorrection(tac, ctps, masks[...,np.newaxis])
    cbf,_,_ = CalcParaMaps.CalcParaMaps(bc, masks, kappa=1, rho=1, aif=np.copy(aif))
    cbf *= cbfFac
    
    rmses = np.sqrt(np.sum((cbf - cbf0)**2 * masks, (1,2)) / np.sum(masks, (1,2)))
#     print (np.mean(rmses))
    
    np.save(os.path.join(os.path.dirname(filename), 'rmse_cbf'), rmses)

0,1,2,3,4,

In [32]:
# Calc all the rmses for Supervised
filenames = glob.glob('/home/dwu/trainData/Noise2Noise/train/ctp/simul/supervised_beta_*_N0_%g/tmp/iodines.npy'%N0)
for i, filename in enumerate(filenames):
    print (i, end=',')
    tac = np.load(filename)[iTests,...].transpose(0, 2, 3, 1) * 0.025 / imgNorm
    bc = BiasCorrection.BiasCorrection(tac, ctps, masks[...,np.newaxis])
    cbf,_,_ = CalcParaMaps.CalcParaMaps(bc, masks, kappa=1, rho=1, aif=np.copy(aif))
    cbf *= cbfFac
    
    rmses = np.sqrt(np.sum((cbf - cbf0)**2 * masks, (1,2)) / np.sum(masks, (1,2)))
    
    np.save(os.path.join(os.path.dirname(filename), 'rmse_cbf'), rmses)

0,1,2,3,4,